<a href="https://colab.research.google.com/github/kankkw/229352-StatisticalLearning/blob/main/Lab08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ปฏิบัติการครั้งที่ 8

ดาวน์โหลดข้อมูลอุณหภูมิสูงสุดรายวันปี 1998-2019 จาก link ข้างล่างนี้  
https://donlapark.pages.dev/229351/data/CM_temp.csv

1. เปลี่ยนจากข้อมูลรายวันให้เป็นข้อมูลค่าเฉลี่ยรายเดือน
2. จงแบ่งข้อมูลโดยให้ Test set เป็นข้อมูลของ 24 เดือนสุดท้ายและ Training set เป็นข้อมูลก่อนหน้าทั้งหมด
3. ลองสร้างโมเดล Holt-Winters แบบต่างๆ (damped, non-damped, additive, multiplicative etc.) แล้วระบุโมเดลที่ดีที่ให้ค่า RMSE บน test set น้อยที่สุด
4. จงแสดงแผนภาพของข้อมูลจริงกับข้อมูลที่ได้ในข้อ 3

In [ ]:
!wget -O CM_temp.csv https://donlapark.pages.dev/229351/data/CM_temp.csv

In [ ]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from datetime import datetime

In [ ]:
# ฟังก์ชันที่เปลี่ยน string ให้เป็น datetime
def parser(dates):
  return datetime.strptime(dates, '%Y-%m-%d')

# parse_dates ชื่อของคอลัมน์ที่จะเปลี่ยนให้เป็น datetime
# index_col ชื่อของคอลัมน์ที่จะให้เป็น index
# date_parser ฟังก์ชันที่เปลี่ยน string ให้เป็น datetime
data = pd.read_csv('CM_temp.csv', parse_dates=['Date'],
                                        index_col='Date',
                                        date_parser=parser)

data.tail()

ข้อมูลที่ได้เป็นข้อมูลรายวัน ทำการหาค่าเฉลี่ยรายเดือนด้วย method `resample`

In [ ]:
data_monthly = data['MaxTemp'].resample("M").mean()

data_monthly.head()

In [ ]:
def rmse(y_true,y_pred):
  # Inputs
  # y_true: actual value, y_pred: predicted values
  # Output: RMSE between y_true and y_pred
  return np.sqrt(np.mean(np.square(y_true-y_pred)))

In [ ]:
results = {}
configs = [
    ('additive', False),
    ('multiplicative', False),
    ('additive', True),
    ('multiplicative', True)
]

for seasonal, damped in configs:
    model = ExponentialSmoothing(train,
                                 trend='add',
                                 seasonal=seasonal,
                                 seasonal_periods=12,
                                 damped_trend=damped).fit()

    pred = model.forecast(24)
    error = rmse(test, pred)
    results[(seasonal, damped)] = (error, model, pred)

best_cfg = min(results, key=lambda x: results[x][0])
best_rmse, best_model, best_pred = results[best_cfg]

print("\nBest Model:", best_cfg)
print("Best RMSE:", best_rmse)

plt.figure(figsize=(12,6))
plt.plot(train.index, train, label='Training')
plt.plot(test.index, test, label='Test', color='orange')
plt.plot(test.index, best_pred, label=f'Forecast ({best_cfg})', color='red')
plt.title('Holt-Winters Forecast vs Actual')
plt.legend()
plt.show()

rmse_summary = pd.DataFrame({
    'Seasonal': [cfg[0] for cfg in results.keys()],
    'Damped': [cfg[1] for cfg in results.keys()],
    'RMSE': [results[cfg][0] for cfg in results.keys()]
}).sort_values(by='RMSE')

print("\nRMSE Summary of All Models:")
print(rmse_summary)